In [1]:
import argparse
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})
from tqdm import tqdm
import struct
import pickle
from scipy.interpolate import RectBivariateSpline
import scipy as sp
import os
import glob
import numpy as np
np.seterr('raise')
from sklearn.linear_model import LinearRegression
import seaborn as sns
np.set_printoptions(precision=4)

%matplotlib notebook

def read_complex_binary2(filename):
    """ Read file of float32 into complex array.
    """

    with open(filename, 'rb') as f:
        bytes = f.read()
    data = np.frombuffer(bytes, dtype=np.float32).reshape(-1, 2)
    data = data[:, 0] + 1j*data[:, 1]
    return data

def get_rssis(filename):
    """ Get RSSI time series from file.
    """
    rssis = []
    data = read_complex_binary2(filename)
    num_frame = int(len(data)/65536 - 2)
    for i in range(num_frame):
        section = data[i*65536:(i+1)*65536]
        spectrum = 10*np.log10(np.fft.fftshift(np.abs(np.fft.fft(section))**2))
        rssis.append(np.max(spectrum[22929:22949]))
    rssis = np.array(rssis)
    return rssis

def get_test_grid(dirname, prefix, xs=None, ys=None):
    """ Get test grid from xs, ys.
    """
    test_rssis = {}
    
    if xs is None or ys is None:
        for filename in tqdm(glob.glob(os.path.join(dirname, f"{prefix}*.dat"))):
            coords = os.path.splitext(os.path.basename(filename))[0][1:]
            x, y = int(coords[:2]), int (coords[2:])
            
            filename_cached = os.path.splitext(filename)[0] + '.pkl'
            if os.path.exists(filename_cached):
                with open(filename_cached, 'rb') as f:
                    mean, std = pickle.load(f)
                test_rssis[x,y] = mean
            else:
                rssis = get_rssis(filename)
                if len(rssis):
                    mean, std = np.mean(rssis), np.std(rssis)
                    with open(filename_cached, 'wb') as f:
                        pickle.dump((mean,std), f)
                    test_rssis[x,y] = mean
                
    else:
        for i, x in enumerate(tqdm(xs)):
            for j, y in enumerate(ys):
                filename = os.path.join(dirname, f"{prefix}{x:>02}{y:>02}.dat")
                filename_cached = os.path.splitext(filename)[0] + '.pkl'
                if os.path.exists(filename_cached):
                    with open(filename_cached, 'rb') as f:
                        mean, std = pickle.load(f)
                    test_rssis[x,y] = mean
                else:
                    rssis = get_rssis(filename)
                    if len(rssis):
                        mean, std = np.mean(rssis), np.std(rssis)
                        with open(filename_cached, 'wb') as f:
                            pickle.dump((mean,std), f)
                        test_rssis[x,y] = mean
    
    return test_rssis

# Linear Regression

In [2]:
def get_model_linear(dirname, prefix, xs, ys):
    """ Get the linear model from directory of data files.
    """
    s_mean = np.full_like(np.meshgrid(xs,ys)[0], np.nan, dtype=np.float64)

    for i, x in enumerate(tqdm(xs)):
        for j, y in enumerate(ys):
            filename = os.path.join(dirname, f"{prefix}{x:>02}{y:>02}.dat")
            filename_cached = os.path.splitext(filename)[0] + '.pkl'
            if os.path.exists(filename_cached):
                with open(filename_cached, 'rb') as f:
                    mean, std = pickle.load(f)
                s_mean[i,j] = mean
            else:
                rssis = get_rssis(filename)
                if len(rssis):
                    mean, std = np.mean(rssis), np.std(rssis)
                    with open(filename_cached, 'wb') as f:
                        pickle.dump((mean,std), f)
                    s_mean[i,j] = mean
    
    X, Y = np.meshgrid(xs,ys)
    A = np.sqrt(X**2+Y**2).flatten().reshape(-1, 1)
    b = s_mean.flatten().reshape(-1, 1)
    rssi2dist = LinearRegression().fit(b, A)

    return rssi2dist

def localize_linear(rssis, rssi2dist, n):
    """ Compute location estimate using maximum-likelihood estimation.
    """
    d1, d2, d3, d4 = [rssi2dist.predict(np.array(rssi).reshape(1, -1)) for rssi in rssis]
    
    xs = np.arange(0, n+0.1, 0.1)
    ys = np.arange(0, n+0.1, 0.1)
    
    error_grid = np.zeros_like(np.meshgrid(xs,ys)[0], dtype=np.float64)
    for i, x in enumerate(xs):
        for j, y in enumerate(ys):
            d1_ = np.sqrt(x**2+y**2)
            d2_ = np.sqrt((n-y)**2 + x**2)
            d3_ = np.sqrt((n-x)**2 + (n-y)**2)
            d4_ = np.sqrt(y**2 + (n-x)**2)
            error_grid[i,j] = (d1-d1_)**2 + (d2-d2_)**2 + (d3-d3_)**2 + (d4-d4_)**2
    
    result = np.unravel_index(np.argmin(error_grid), error_grid.shape)
    
    return result[0]/10.0, result[1]/10.0

def run_lr(dirname, n,
           train_prefix, test_prefix, 
           train_xs, train_ys, test_xs, test_ys):
    
    test_grid = get_test_grid(dirname, test_prefix, test_xs, test_ys)
    rssi2dist = get_model_linear(dirname, train_prefix, train_xs, train_ys)

    # Start testing
    errors = []
    prediction_record=[]
    for (x,y) in tqdm(test_grid.keys()):
        rssis = [test_grid[x, y], 
                 test_grid[n-y, x],
                 test_grid[n-x, n-y], 
                 test_grid[y, n-x]]
        x_pred, y_pred = localize_linear(rssis, rssi2dist, n)
        errors.append(np.sqrt((x-x_pred)**2+(y-y_pred)**2))
        prediction_record.append([x,y,x_pred,y_pred])
    print(f"mean error: {np.mean(errors)}")
    print(f"median error: {np.median(errors)}")
    print(f"stdev error: {np.std(errors)}")
    
    return errors, prediction_record

In [4]:
errors_lr = []

# Dataset
# dirname = 'data/Sep 17- redo water'
# n = 10 # grid is 0 to n (n+1 by n+1)
# # AEM*
# train_prefix = 0
# train_xs = np.arange(0,n+1,2)
# train_ys = np.arange(0,n+1,2)
# test_prefix = 1
# # Run prediction
# errors, prediction_record = run_lr(dirname, n,
#                                    train_prefix, test_prefix,
#                                    train_xs, train_ys, None, None)
# errors_lr += errors

# Dataset
dirname = 'data/sep-18-first'
n = 10 # grid is 0 to n (n+1 by n+1)
# AEM*
train_prefix = 0
train_xs = np.arange(0,n+1,2)
train_ys = np.arange(0,n+1,2)
test_prefix = 1
# Run prediction
errors, prediction_record = run_lr(dirname, n,
                                   train_prefix, test_prefix,
                                   train_xs, train_ys, None, None)
errors_lr += errors

# Dataset
dirname = 'data/sep-19-full'
n = 10 # grid is 0 to n (n+1 by n+1)
# AEM*
train_prefix = 0
train_xs = np.arange(0,n+1,2)
train_ys = np.arange(0,n+1,2)
test_prefix = 1
# Run prediction
errors, prediction_record = run_lr(dirname, n,
                                   train_prefix, test_prefix,
                                   train_xs, train_ys, None, None)

errors_lr += errors

# Dataset
dirname = 'data/sep-20-redo'
n = 10 # grid is 0 to n (n+1 by n+1)
# AEM*
train_prefix = 0
train_xs = np.arange(0,n+1,2)
train_ys = np.arange(0,n+1,2)
test_prefix = 1
# Run prediction
errors, prediction_record = run_lr(dirname, n,
                                   train_prefix, test_prefix,
                                   train_xs, train_ys, None, None)
errors_lr += errors

with open('errors_lr.pkl', 'wb') as f:
    pickle.dump(errors_lr, f)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:04<00:00,  7.23it/s]


mean error: 2.8624626117782066
median error: 3.047950130825634
stdev error: 0.8430612318309594


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:08<00:00,  7.09it/s]


mean error: 3.4654428132067077
median error: 3.5128336140500593
stdev error: 1.2452735074649208


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:03<00:00,  7.16it/s]

mean error: 3.4020600310680744
median error: 3.6674241641784495
stdev error: 1.29197041181642
